In [62]:
import json
import requests
import datetime as dt
import pickle
import pandas as pd
import mysql.connector
import os
from os.path import join, dirname
from datetime import date, datetime

from dotenv import load_dotenv
import db_helper

In [63]:
def get_odds_data():
    api_key = 'da3195c8356b3b0ee3e5a6682ed9c537'
    sports_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
        'api_key': api_key,
        'sport': 'basketball_ncaab',
        'region': 'us',
        'oddsFormat': 'american'
    })


#     print(sports_response.headers['x-requests-remaining'])
    sports_json = json.loads(sports_response.text)
    return sports_json
    

In [64]:
print("https://api.the-odds-api.com/v3/odds/?apiKey=da3195c8356b3b0ee3e5a6682ed9c537&sport=basketball_ncaab&region=us&mkt=h2h&oddsFormat=american")

https://api.the-odds-api.com/v3/odds/?apiKey=da3195c8356b3b0ee3e5a6682ed9c537&sport=basketball_ncaab&region=us&mkt=h2h&oddsFormat=american


In [65]:
def getTeamData(teamid):
    teamid = teamid.replace(" ", "%20")
    baseUrl="https://fastbreakstats-server-main-hqc7w3njda-uc.a.run.app/teams/"
    url=baseUrl+teamid

    r = requests.get(url = url).json()
    bpival = r['bpi']
    kpval = r['kp']
    srsval = r['srs']
    return kpval, bpival, srsval

In [66]:
def get_needed_odds(conf):
    #This function calculates the odds needed for a bet to have an expected value of .1
    #An EV of .1 means a $.10 expected return on $1 over time
    #The normal formula is EV = (payout * confidence) - (bet * (1-conf))
    #payout is the expected profit on a bet.
    #confidence is models % chance of winning. 1-conf is for the other team
    #The bet is always going to be $1 for simple math which removes the need for the bet variable
    #below equation is rearranged to isolate and solve for the payout
    payout = round(((.1+(1-conf))/conf),4) 
    #convert payout to american odds
    if 100/payout > 100:
        odds = round(-100/payout)
    else:
        odds = round(100*payout)
    return odds

In [67]:
def get_ev(odds,conf):
    #This calculates the Expected Return on a $1 bet with the actual given bookie odds we pass in
    #EV of .1 means a 10 cent return on a $1 bet over time
    if int(odds) > 0:
        payout = odds/100
    else:
        payout = -100/odds
    ev = (conf * payout) - (1-conf)
    return ev

In [68]:
def get_kelly_bet(odds,conf):
    if int(odds) > 0:
        dec = 1+(odds/100)
    else:
        dec = 1-(100/odds)
    kb = ((dec*conf) - (1-conf))/dec
    return kb

In [69]:
def convert_epoch(epoch):
    return dt.datetime.fromtimestamp(epoch).strftime('%Y-%m-%d %H:%M:00')

In [77]:
def get_all_good_odds(teams,loaded_model,sports_json):
    games_to_bet = []
    for game in sports_json['data']:
        #Loop through and get odds
        if game['commence_time'] < dt.datetime.now().timestamp():
            continue
        if dt.datetime.fromtimestamp(game['commence_time']).date() > dt.datetime.now().date():
            continue

        game_time = convert_epoch(game['commence_time'])
        home = game['home_team']
        if game['home_team'] == game['teams'][0]:
            away = game['teams'][1]
        else:
            away = game['teams'][0]

        try:
            home = home.replace("-", " ")
            home = home.replace("(", "")
            home = home.replace(")", "")
            home = home.replace(".", "")
            away = away.replace("-", " ")
            away = away.replace("(", "")
            away = away.replace(")", "")
            away = away.replace(".", "")
            homeid = str(teams.loc[teams['odds_api_name'] == home]['teamID'].values[0])
            awayid = str(teams.loc[teams['odds_api_name'] == away]['teamID'].values[0])
            homekp, homebpi, homesrs = getTeamData(homeid)
            awaykp, awaybpi, awaysrs = getTeamData(awayid)
        except Exception as e:
            print(e)
            print(game['teams'])
            continue

        res = loaded_model.predict_proba([[float(awaykp), float(awaybpi), float(awaysrs), float(homekp), float(homebpi), float(homesrs)]])
        away_needed_odds = int(get_needed_odds(res[0][0]))    
        home_needed_odds = int(get_needed_odds(res[0][1]))

        #site,odds,update,ev
        bovada = ['bovada',None,None]
        draftkings = ['dk',None,None]
        willhill = ['wh',None,None]
    #     print()
        sites = list(filter(lambda site: site['site_key'] in ['williamhill_us','draftkings','bovada'], game['sites']))
    #     print(sites)
        if not sites:
            continue
        for site in sites:
            if site['site_key'] == 'williamhill_us':
                willhill[2] = convert_epoch(site['last_update'])
                if home == game['teams'][0]:
                    home_odds = int(site['odds']['h2h'][0])
                    away_odds = int(site['odds']['h2h'][1])
                else:
                    home_odds = int(site['odds']['h2h'][1])
                    away_odds = int(site['odds']['h2h'][0])
                if home_odds > home_needed_odds:
                    willhill[1] = home_odds
                    single_needed_odds = home_needed_odds
                    team_to_bet = homeid
#                     willhill[3] = round(get_ev(home_odds,res[0][1]),2)
                if away_odds > away_needed_odds:
                    willhill[1] = away_odds
                    single_needed_odds = away_needed_odds
                    team_to_bet = awayid
#                     willhill[3] = round(get_ev(away_odds,res[0][0]),2)
            elif site['site_key'] == 'draftkings':
                draftkings[2] = convert_epoch(site['last_update'])
                if home == game['teams'][0]:
                    home_odds = int(site['odds']['h2h'][0])
                    away_odds = int(site['odds']['h2h'][1])
                else:
                    home_odds = int(site['odds']['h2h'][1])
                    away_odds = int(site['odds']['h2h'][0])
                if home_odds > home_needed_odds:
                    draftkings[1] = home_odds
                    single_needed_odds = home_needed_odds
                    team_to_bet = homeid
#                     draftkings[3] = round(get_ev1(home_odds,res[0][1]),2)
                if away_odds > away_needed_odds:
                    draftkings[1] = away_odds
                    single_needed_odds = away_needed_odds
                    team_to_bet = awayid
#                     draftkings[3] = round(get_ev1(away_odds,res[0][0]),2)
            elif site['site_key'] == 'bovada':
                bovada[2] = convert_epoch(site['last_update'])
                if home == game['teams'][0]:
                    home_odds = int(site['odds']['h2h'][0])
                    away_odds = int(site['odds']['h2h'][1])
                else:
                    home_odds = int(site['odds']['h2h'][1])
                    away_odds = int(site['odds']['h2h'][0])
                if home_odds > home_needed_odds:
                    bovada[1] = home_odds
                    single_needed_odds = home_needed_odds
                    team_to_bet = homeid
#                     bovada[3] = round(get_ev1(home_odds,res[0][1]),2)
                if away_odds > away_needed_odds:
                    bovada[1] = away_odds
                    single_needed_odds = away_needed_odds
                    team_to_bet = awayid
#                     bovada[3] = round(get_ev1(away_odds,res[0][0]),2)

        if bovada[1] or draftkings[1] or willhill[1]:
    #         print(game['teams'],game['home_team'])
    #         print("Game Time:", game_time)
    #         print('Home team:', homeid)
    #         print('Away team:', awayid)
    #         print('Team to bet:', team_to_bet)
    #         print('Needed odds:', single_needed_odds)
    #         print(bovada)
    #         print(draftkings)
    #         print(willhill)
            game_to_bet = []
            game_to_bet.append(game_time)
            game_to_bet.append(str(homeid))
            game_to_bet.append(str(awayid))
            game_to_bet.append(str(bovada[1]))
            game_to_bet.append(str(draftkings[1]))
            game_to_bet.append(str(willhill[1]))
            game_to_bet.append(bovada[2])
            game_to_bet.append(draftkings[2])
            game_to_bet.append(willhill[2])
            game_to_bet.append(str(single_needed_odds))
            game_to_bet.append(str(team_to_bet))
            
            games_to_bet.append(game_to_bet)
        else:
            continue

    return games_to_bet

In [78]:
url="https://fastbreakstats-server-main-hqc7w3njda-uc.a.run.app/teams"
r = requests.get(url = url).json()
teams = pd.DataFrame(r)
loaded_model = pickle.load(open('6yr_LR.sav', 'rb'))
# games_to_bet = []
sports_json = get_odds_data()

c:\users\blake\appdata\local\programs\python\python36\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [79]:
good_odds = get_all_good_odds(teams,loaded_model,sports_json)

index 0 is out of bounds for axis 0 with size 0
["Hawai'i Rainbow Warriors", 'UC Riverside Highlanders']


In [80]:
for bet in good_odds:
    print(bet)

['2021-01-08 15:00:00', '240', '158', '-150', '-148', '-145', '2021-01-08 10:28:00', '2021-01-08 10:27:00', '2021-01-08 10:27:00', '-279', '240']
['2021-01-08 15:00:00', '198', '245', '-290', 'None', 'None', '2021-01-08 10:28:00', None, None, '-325', '198']
['2021-01-08 16:00:00', '107', '218', 'None', '112', '105', None, '2021-01-08 10:27:00', '2021-01-08 10:27:00', '-136', '107']
['2021-01-08 17:00:00', '10', '101', '-145', '-148', '-140', '2021-01-08 10:28:00', '2021-01-08 10:27:00', '2021-01-08 10:27:00', '-193', '10']
['2021-01-08 17:30:00', '301', '219', 'None', '-175', '-170', None, '2021-01-08 10:27:00', '2021-01-08 10:27:00', '-252', '301']
['2021-01-08 18:00:00', '223', '90', 'None', '-360', 'None', '2021-01-08 10:28:00', '2021-01-08 10:27:00', '2021-01-08 10:27:00', '-374', '223']
['2021-01-08 18:00:00', '172', '238', 'None', '180', '190', None, '2021-01-08 10:27:00', '2021-01-08 10:27:00', '161', '238']
['2021-01-08 18:00:00', '54', '214', 'None', '-143', '-130', None, '202

In [74]:
db = db_helper.Db_Helper()

In [75]:
db.insert_odds(good_odds)

In [76]:
db.close()